# Самостоятельное задание

In [1]:
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
from IPython.display import HTML
%matplotlib notebook

In [2]:
from kinematics import Vector, Quaternion, Transform
import graphics

SyntaxError: invalid syntax (<ipython-input-3-81538c60c443>, line 3)

In [4]:
## Решение прямой задачи кинематики для манипулятора PUMA

Манипулятор кинематической схемы PUMA обладает шестью степенями подвижности.

![внешний виж PUMA](fig/puma_model.png)

Его можно условно разделить на сегменты, соединияюще между собой:
- основание
- плечо
- локоть
- кисть
- фланец

![кинематическая схема PUMA](https://drive.google.com/file/d/1FxugUzcoZWfQ7uiiimD8SeVNj6ZS6rj-/view?usp=sharing)


SyntaxError: invalid syntax (<ipython-input-4-b436a5d06180>, line 3)

## Задание

Для своего манипулятора решите прямую задачу кинематики.

In [ ]:
def UR_chain(q, l):
    base = Transform.identity()
    #q1
    shoulder = base + Transform(
    Vector(0, 0, l[0]),
    Quaternion.from_angle_axis(q[0], Vector(0, 0, 1), math_source)
    )
    #q2
    elbow = shoulder + Transform(
        Vector(0, l[1] + l[4], 0 ),
        Quaternion.from_angle_axis(q[1], Vector(0, 1, 0), math_source)
    )
    #q3
    elbow2 = elbow + Transform(
    Vector(0, 0,  l[2] ),
    Quaternion.from_angle_axis(q[2], Vector(0, 1, 0), math_source)
    )
    #q4
    wrist = elbow2 + Transform(
        Vector(0, 0, l[3]),
        Quaternion.from_angle_axis(q[3], Vector(0, 1, 0), math_source) 
    )
    #q5
    flange = wrist + Transform(
        Vector(0,0, l[5]),
        Quaternion.from_angle_axis(q[4], Vector(0, 0, 1), math_source)
    )
    #q6
    end = base + Transform(
        Vector(0, 0, 0),
        Quaternion.from_angle_axis(q[5], Vector(0, 0, 1))
    )
    return [
        base,shoulder,elbow,elbow2,
        end
    ]

Задайте закон изменения обобщенных координат:

In [ ]:
def UR_q(t, total):
    #omega = t / total * np.pi * 2
    a = (t - total) / 2
    omega = (-np.cos(3.14 / total * t) + 1.0)*5

    return [
        np.pi / 4 * np.cos(omega),
        np.pi / 6* np.sin(omega),
        np.pi / 2,
        np.pi*3/4,
        np.pi *np.sin(omega)/ 6,
        0
    ]

Укажите длину звеньев:

In [ ]:
custom_l = [0]

In [ ]:
custom_fig = plt.figure()
ax = custom_fig.add_subplot(projection="3d")
ax.set_xlim([-2, 2]); ax.set_ylim([-2, 2]), ax.set_zlim([0, 4])
lines, = ax.plot([], [], [], color="#000000")
graphics.axis(ax, Transform.identity(), 2);
r, g, b = graphics.axis(ax, Transform.identity())

total = 100

def animate(frame):
    chain = custom_chain(custom_q(frame, total), custom_l)
    (x, y, z) = graphics.chain_to_points(chain)
    lines.set_data_3d(x, y, z)
    global r, g, b
    r.remove(); g.remove(); b.remove()
    r, g, b = graphics.axis(ax, chain[-1], 0.5)

animate(0)
fps = 25
custom_ani = animation.FuncAnimation(
    custom_fig,
    animate,
    frames=total,
    interval=1000.0/fps
)

In [ ]:
HTML(custom_ani.to_jshtml())

## Отображение траектории

In [ ]:
t_end = 10
step = 0.1
t = np.arange(0, t_end, step)
chain = custom_chain(custom_q(t, t_end), custom_l)
x, y, z = graphics.chain_to_points(chain)

fig = plt.figure()
ax = fig.add_subplot(projection="3d")
ax.set_xlim([-2, 2]); ax.set_ylim([-2, 2]); ax.set_zlim([0, 4])
ax.plot(x[-1], y[-1], z[-1], color="#000000")
fig.show()

## Измерение скорости движения

Численно продиффиринцируем положение всех точек траектории

In [ ]:
t_end = 10
step = 0.01
t = np.arange(0, t_end, step)
chain = custom_chain(custom_q(t, t_end), custom_l)
end = chain[-1]
velocity_x = np.diff(end.translation.x) / step
velocity_y = np.diff(end.translation.y) / step
velocity_z = np.diff(end.translation.z) / step
velocity_total = (velocity_x ** 2 + velocity_y ** 2 + velocity_z ** 2) ** 0.5

fig = plt.figure()
ax = fig.add_subplot()

ax.plot(t[:-1], velocity_x, color="#ff0000", label="$V_x$")
ax.plot(t[:-1], velocity_y, color="#00ff00", label="$V_y$")
ax.plot(t[:-1], velocity_z, color="#0000ff", label="$V_z$")
ax.plot(t[:-1], velocity_total, color="#000000", label="$V$")
fig.legend()
fig.show()

## Самостоятельные задания

Для своего закона изменения обобщенных координат

- Оцените диапазон $X$, $Y$ и $Z$ координат для данной траектории;
- Оцените максимальную скорость по осям;
- Оцените абсолютную максимальную скорость.